In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 14:04:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
l = [('X', )]

In [4]:
df = spark.createDataFrame(l,schema = """dummy STRING""")

In [5]:
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [6]:
df.printSchema()

root
 |-- dummy: string (nullable = true)



In [7]:
df.select(split(lit("Hello, How are you?")," ").alias('Splitted')[1]).show(truncate = False)

+------------------------------------------------+
|split(Hello, How are you?,  , -1) AS Splitted[1]|
+------------------------------------------------+
|How                                             |
+------------------------------------------------+



In [8]:
df.select(explode(split(lit("There are 5 pegions on the branch of Mango Tree")," ")).alias('exploded')).show()

+--------+
|exploded|
+--------+
|   There|
|     are|
|       5|
| pegions|
|      on|
|     the|
|  branch|
|      of|
|   Mango|
|    Tree|
+--------+



In [9]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890,+1 234 567 8901", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111,+44 222 222 2222", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210,+61 876 543 2109", "789 12 6118"
                     )
                ]

In [10]:
employeesDF = spark.createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_numbers STRING, ssn STRING"""
                   )

In [11]:
employeesDF. \
    select('employee_id', 'phone_numbers'). \
    show(truncate=False)

+-----------+---------------------------------+
|employee_id|phone_numbers                    |
+-----------+---------------------------------+
|1          |+1 123 456 7890,+1 234 567 8901  |
|2          |+91 234 567 8901                 |
|3          |+44 111 111 1111,+44 222 222 2222|
|4          |+61 987 654 3210,+61 876 543 2109|
+-----------+---------------------------------+



In [12]:
empDF = employeesDF. \
    select('employee_id','phone_numbers','ssn'). \
    withColumn('distinct numbers', explode(split('phone_numbers', ',')))

In [13]:
empDF.show()

+-----------+--------------------+-----------+----------------+
|employee_id|       phone_numbers|        ssn|distinct numbers|
+-----------+--------------------+-----------+----------------+
|          1|+1 123 456 7890,+...|123 45 6789| +1 123 456 7890|
|          1|+1 123 456 7890,+...|123 45 6789| +1 234 567 8901|
|          2|    +91 234 567 8901|456 78 9123|+91 234 567 8901|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1111|
|          3|+44 111 111 1111,...|222 33 4444|+44 222 222 2222|
|          4|+61 987 654 3210,...|789 12 6118|+61 987 654 3210|
|          4|+61 987 654 3210,...|789 12 6118|+61 876 543 2109|
+-----------+--------------------+-----------+----------------+



In [16]:
empDF. \
    select('employee_id','distinct numbers','ssn'). \
    withColumn('area code',split('distinct numbers',' ')[1].cast('int')). \
    withColumn('phone last 4',split('distinct numbers',' ')[3].cast('int')). \
    withColumn('ssn last 4',split('ssn',' ')[2].cast('int')).printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- distinct numbers: string (nullable = false)
 |-- ssn: string (nullable = true)
 |-- area code: integer (nullable = true)
 |-- phone last 4: integer (nullable = true)
 |-- ssn last 4: integer (nullable = true)



In [17]:
empDF. \
    select('employee_id','distinct numbers','ssn'). \
    withColumn('area code',split('distinct numbers',' ')[1].cast('int')). \
    withColumn('phone last 4',split('distinct numbers',' ')[3].cast('int')). \
    withColumn('ssn last 4',split('ssn',' ')[2].cast('int')).show(truncate = False)

+-----------+----------------+-----------+---------+------------+----------+
|employee_id|distinct numbers|ssn        |area code|phone last 4|ssn last 4|
+-----------+----------------+-----------+---------+------------+----------+
|1          |+1 123 456 7890 |123 45 6789|123      |7890        |6789      |
|1          |+1 234 567 8901 |123 45 6789|234      |8901        |6789      |
|2          |+91 234 567 8901|456 78 9123|234      |8901        |9123      |
|3          |+44 111 111 1111|222 33 4444|111      |1111        |4444      |
|3          |+44 222 222 2222|222 33 4444|222      |2222        |4444      |
|4          |+61 987 654 3210|789 12 6118|987      |3210        |6118      |
|4          |+61 876 543 2109|789 12 6118|876      |2109        |6118      |
+-----------+----------------+-----------+---------+------------+----------+

